In [2]:
#Trần Lương Tiến Sĩ
#19522128
#github: https://github.com/tranluongtiensi/datamining

IN CLASS


I. Feature Engineering
1. Text Normalization 

In [ ]:
import numpy as np
import pandas as pd
data = pd.read_csv('elonmusk_tweets.csv')
print(len(data))
data.head()

2819


,id,created_at,text
0,849636868052275200,2017-04-05 14:56:29,b'And so the robots spared humanity ... https:...
1,848988730585096192,2017-04-03 20:01:01,"b""@ForIn2020 @waltmossberg @mims @defcon_5 Exa..."
2,848943072423497728,2017-04-03 16:59:35,"b'@waltmossberg @mims @defcon_5 Et tu, Walt?'"
3,848935705057280001,2017-04-03 16:30:19,b'Stormy weather in Shortville ...'
4,848416049573658624,2017-04-02 06:05:23,"b""@DaveLeeBBC @verge Coal is dying due to nat ..."


In [ ]:
from __future__ import print_function, division
from nltk.stem import PorterStemmer, WordNetLemmatizer
import nltk
nltk.download('punkt')
import string
from nltk.corpus import stopwords
import math
from collections import Counter
nltk.download('stopwords')
import pprint 
pp = pprint.PrettyPrinter(indent=4)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
def normalize(document):
    # TODO: remove punctuation
    text = "".join([ch for ch in document if ch not in string.punctuation])
    
    # TODO: tokenize text
    tokens = nltk.word_tokenize(text)
    
    # TODO: Stemming
    stemmer = PorterStemmer()
    ret = " ".join([stemmer.stem(word.lower()) for word in tokens])
    return ret

original_documents = [x.strip() for x in data['text']] 
documents = [normalize(d).split() for d in original_documents]
documents[0]

['band', 'so', 'the', 'robot', 'spare', 'human', 'httpstcov7jujqwfcv']

2. Implement TF-IDF

In [ ]:
# Flatten all the documents
flat_list = [word for doc in documents for word in doc]

# TODO: remove stop words from the vocabulary
words = [word for word in flat_list if word not in stopwords.words('english')]

# TODO: we take the 500 most common words only
counts = Counter(words)
vocabulary = counts.most_common(500)
print([x for x in vocabulary if x[0] == 'tesla'])
vocabulary = [x[0] for x in vocabulary]
assert len(vocabulary) == 500

# vocabulary.sort()
vocabulary[:5]

[('tesla', 287)]


['brt', 'tesla', 'spacex', 'model', 'thi']

In [ ]:
def tf(vocabulary, documents):
    matrix = [0] * len(documents)
    for i, document in enumerate(documents):
        counts = Counter(document)
        matrix[i] = [0] * len(vocabulary)
        for j, term in enumerate(vocabulary):
            matrix[i][j] = counts[term]
    return matrix

tf = tf(vocabulary, documents)
np.array(vocabulary)[np.where(np.array(tf[1]) > 0)], np.array(tf[1])[np.where(np.array(tf[1]) > 0)]

(array(['tesla', 'exactli'], dtype='<U17'), array([1, 1]))

In [ ]:
def idf(vocabulary, documents):
    """TODO: compute IDF, storing values in a dictionary"""
    idf = {}
    num_documents = len(documents)
    for i, term in enumerate(vocabulary):
      idf[term] = math.log(num_documents / sum(term in document for document in documents), 2)
    return idf

idf = idf(vocabulary, documents)
[idf[key] for key in vocabulary[:5]]

[2.539126825495932,
 3.3163095197385393,
 3.7262581423445837,
 3.8171115727956972,
 3.8027562798186274]

In [ ]:
def vectorize(document, vocabulary, idf):
    vector = [0]*len(vocabulary)
    counts = Counter(document)
    for i,term in enumerate(vocabulary):
        vector[i] = idf[term] * counts[term]
    return vector

document_vectors = [vectorize(s, vocabulary, idf) for s in documents]
np.array(vocabulary)[np.where(np.array(document_vectors[1]) > 0)], np.array(document_vectors[1])[np.where(np.array(document_vectors[1]) > 0)]

(array(['tesla', 'exactli'], dtype='<U17'), array([3.31630952, 6.65361284]))

### 1.3. Compare the results with the reference implementation of scikit-learn library.
Now we use the scikit-learn library. As you can see that, the way we do text normalization affects the result. Feel free to further improve upon (OPTIONAL), e.g. https://stackoverflow.com/questions/36182502/add-stemming-support-to-countvectorizer-sklearn

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import linear_kernel

tfidf = TfidfVectorizer(analyzer='word', ngram_range=(1,1), min_df = 1, stop_words = 'english', max_features=500)

features = tfidf.fit(original_documents)
corpus_tf_idf = tfidf.transform(original_documents) 

sum_words = corpus_tf_idf.sum(axis=0)
words_freq = [(word, sum_words[0, idx]) for word, idx in tfidf.vocabulary_.items()]
print(sorted(words_freq, key = lambda x: x[1], reverse=True)[:5])
print('testla', corpus_tf_idf[1, features.vocabulary_['tesla']])

[('http', 163.54366542841234), ('https', 151.85039944652075), ('rt', 112.61998731390989), ('tesla', 95.96401470715628), ('xe2', 88.20944486346477)]
testla 0.3495243100660956


### 1.4.  Apply TF-IDF for information retrieval
We can use the vector representation of documents to implement an information retrieval system. We test with the query $Q$ = "tesla nasa"

In [ ]:
def cosine_similarity(v1,v2):
    """TODO: compute cosine similarity"""
    sumxx, sumxy, sumyy = 0, 0, 0
    for i in range(len(v1)):
      x = v1[i]; y = v2[i]
      sumxx += x*x
      sumyy += y*y
      sumxy += x*y
    if sumxy == 0:
        result = 0
    else:
        result = sumxy/math.sqrt(sumxx*sumyy)
    return result

def search_vec(query, k, vocabulary, stemmer, document_vectors, original_documents):
    q = query.split()
    q = [stemmer.stem(w) for w in q]
    query_vector = vectorize(q, vocabulary, idf)
    
    # TODO: rank the documents by cosine similarity
    scores = [[cosine_similarity(query_vector, document_vectors[d]), d] for d in range(len(document_vectors))]
    scores.sort(key=lambda x: -x[0])
    
    print('Top-{0} documents'.format(k))
    for i in range(k):
        print(i, original_documents[scores[i][1]])

query = "tesla nasa"
stemmer = PorterStemmer()
search_vec(query, 5, vocabulary, stemmer, document_vectors, original_documents)

Top-5 documents
0 b'@ashwin7002 @NASA @faa @AFPAA We have not ruled that out.'
1 b'RT @NASA: Updated @SpaceX #Dragon #ISS rendezvous times: NASA TV coverage begins Sunday at 3:30amET: http://t.co/qrm0Dz4jPE. Grapple at  ...'
2 b"Deeply appreciate @NASA's faith in @SpaceX. We will do whatever it takes to make NASA and the American people proud."
3 b'Would also like to congratulate @Boeing, fellow winner of the @NASA commercial crew program'
4 b"@astrostephenson We're aiming for late 2015, but NASA needs to have overlapping capability to be safe. Would do the same"


In [ ]:
new_features = tfidf.transform([query])

cosine_similarities = linear_kernel(new_features, corpus_tf_idf).flatten()
related_docs_indices = cosine_similarities.argsort()[::-1]

topk = 5
print('Top-{0} documents'.format(topk))
for i in range(topk):
    print(i, original_documents[related_docs_indices[i]])

Top-5 documents
0 b'@ashwin7002 @NASA @faa @AFPAA We have not ruled that out.'
1 b"SpaceX could not do this without NASA. Can't express enough appreciation. https://t.co/uQpI60zAV7"
2 b'@NASA launched a rocket into the northern lights http://t.co/tR2cSeMV'
3 b'Whatever happens today, we could not have done it without @NASA, but errors are ours alone and me most of all.'
4 b'RT @NASA: Updated @SpaceX #Dragon #ISS rendezvous times: NASA TV coverage begins Sunday at 3:30amET: http://t.co/qrm0Dz4jPE. Grapple at  ...'


II. Text Processing

1. Preprocessing

In [ ]:
# Import NLTK and all the needed libraries
import nltk
nltk.download('punkt') #Run this line one time to get the resource
nltk.download('stopwords') #Run this line one time to get the resource
nltk.download('wordnet') #Run this line one time to get the resource
nltk.download('averaged_perceptron_tagger') #Run this line one time to get the resource
import numpy as np
import pandas as pd

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


In [ ]:
# TODO: Load the dataset in coldplay.csv
data = pd.read_csv('coldplay.csv')
data.head(10)

,Artist,Song,Link,Lyrics
0,Coldplay,Another's Arms,/c/coldplay/anothers+arms_21079526.html,Late night watching tv \nUsed to be you here ...
1,Coldplay,Bigger Stronger,/c/coldplay/bigger+stronger_20032648.html,I want to be bigger stronger drive a faster ca...
2,Coldplay,Daylight,/c/coldplay/daylight_20032625.html,"To my surprise, and my delight \nI saw sunris..."
3,Coldplay,Everglow,/c/coldplay/everglow_21104546.html,"Oh, they say people come \nThey say people go..."
4,Coldplay,Every Teardrop Is A Waterfall,/c/coldplay/every+teardrop+is+a+waterfall_2091...,"I turn the music up, I got my records on \nI ..."
5,Coldplay,Everything's Not Lost,/c/coldplay/everythings+not+lost_20032638.html,When I'm counting up my demons \nSaw there wa...
6,Coldplay,Fix You,/c/coldplay/fix+you_10069035.html,When you try your best but you don't succeed ...
7,Coldplay,For You,/c/coldplay/for+you_20032655.html,If you're lost and feel alone \nCircumnavigat...
8,Coldplay,Fun,/c/coldplay/fun_21104545.html,I know it's over before she says \nI know the...
9,Coldplay,Ghost Story,/c/coldplay/ghost+story_21083666.html,Maybe I'm just a ghost \nDisappear when anybo...


In [ ]:
# TODO: Explore the data

import pandas as pd

# Create a DataFrame
df = pd.read_csv('coldplay.csv')

# Print the summary
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Artist  120 non-null    object
 1   Song    120 non-null    object
 2   Link    120 non-null    object
 3   Lyrics  120 non-null    object
dtypes: object(4)
memory usage: 3.9+ KB
None


In [ ]:
# TODO: Select the song 'Every Teardrop Is A Waterfall'
song_row = df[df['Song'] == 'Every Teardrop Is A Waterfall']

# Get the lyrics text for the selected song and save it into a variable
lyrics = song_row['Lyrics'].values[0]

# Print the lyrics
print(lyrics)

I turn the music up, I got my records on  
I shut the world outside until the lights come on  
Maybe the streets alight, maybe the trees are gone  
I feel my heart start beating to my favourite song  
  
And all the kids they dance, all the kids all night  
Until Monday morning feels another life  
I turn the music up  
I'm on a roll this time  
And heaven is in sight  
  
I turn the music up, I got my records on  
From underneath the rubble sing a rebel song  
Don't want to see another generation drop  
I'd rather be a comma than a full stop  
  
Maybe I'm in the black, maybe I'm on my knees  
Maybe I'm in the gap between the two trapezes  
But my heart is beating and my pulses start  
Cathedrals in my heart  
  
As we saw oh this light I swear you, emerge blinking into  
To tell me it's alright  
As we soar walls, every siren is a symphony  
And every tear's a waterfall  
Is a waterfall  
Oh  
Is a waterfall  
Oh oh oh  
Is a is a waterfall  
Every tear  
Is a waterfall  
Oh oh oh  


In [ ]:
# TODO: Tokenize the lyrics of the song and save the tokens into a variable and print it

# Select the row for the song 'Every Teardrop Is A Waterfall'
song_row = df[df['Song'] == 'Every Teardrop Is A Waterfall']

# Get the lyrics text for the selected song and save it into a variable
lyrics = song_row['Lyrics'].values[0]

# Tokenize the lyrics and save the tokens into a variable
tokens = nltk.word_tokenize(lyrics)

# Print the tokens
print(tokens)

['I', 'turn', 'the', 'music', 'up', ',', 'I', 'got', 'my', 'records', 'on', 'I', 'shut', 'the', 'world', 'outside', 'until', 'the', 'lights', 'come', 'on', 'Maybe', 'the', 'streets', 'alight', ',', 'maybe', 'the', 'trees', 'are', 'gone', 'I', 'feel', 'my', 'heart', 'start', 'beating', 'to', 'my', 'favourite', 'song', 'And', 'all', 'the', 'kids', 'they', 'dance', ',', 'all', 'the', 'kids', 'all', 'night', 'Until', 'Monday', 'morning', 'feels', 'another', 'life', 'I', 'turn', 'the', 'music', 'up', 'I', "'m", 'on', 'a', 'roll', 'this', 'time', 'And', 'heaven', 'is', 'in', 'sight', 'I', 'turn', 'the', 'music', 'up', ',', 'I', 'got', 'my', 'records', 'on', 'From', 'underneath', 'the', 'rubble', 'sing', 'a', 'rebel', 'song', 'Do', "n't", 'want', 'to', 'see', 'another', 'generation', 'drop', 'I', "'d", 'rather', 'be', 'a', 'comma', 'than', 'a', 'full', 'stop', 'Maybe', 'I', "'m", 'in', 'the', 'black', ',', 'maybe', 'I', "'m", 'on', 'my', 'knees', 'Maybe', 'I', "'m", 'in', 'the', 'gap', 'betwe

In [ ]:
# TODO: Remove the punctuation, then save the result into a variable and print it

import string
# Select the row for the song 'Every Teardrop Is A Waterfall'
song_row = df[df['Song'] == 'Every Teardrop Is A Waterfall']

# Get the lyrics text for the selected song and save it into a variable
lyrics = song_row['Lyrics'].values[0]

# Remove the punctuation from the lyrics
lyrics_no_punct = lyrics.translate(str.maketrans("", "", string.punctuation))

# Tokenize the lyrics without punctuation and save the tokens into a variable
tokens = nltk.word_tokenize(lyrics_no_punct)

# Print the tokens without punctuation
print(tokens)

['I', 'turn', 'the', 'music', 'up', 'I', 'got', 'my', 'records', 'on', 'I', 'shut', 'the', 'world', 'outside', 'until', 'the', 'lights', 'come', 'on', 'Maybe', 'the', 'streets', 'alight', 'maybe', 'the', 'trees', 'are', 'gone', 'I', 'feel', 'my', 'heart', 'start', 'beating', 'to', 'my', 'favourite', 'song', 'And', 'all', 'the', 'kids', 'they', 'dance', 'all', 'the', 'kids', 'all', 'night', 'Until', 'Monday', 'morning', 'feels', 'another', 'life', 'I', 'turn', 'the', 'music', 'up', 'Im', 'on', 'a', 'roll', 'this', 'time', 'And', 'heaven', 'is', 'in', 'sight', 'I', 'turn', 'the', 'music', 'up', 'I', 'got', 'my', 'records', 'on', 'From', 'underneath', 'the', 'rubble', 'sing', 'a', 'rebel', 'song', 'Dont', 'want', 'to', 'see', 'another', 'generation', 'drop', 'Id', 'rather', 'be', 'a', 'comma', 'than', 'a', 'full', 'stop', 'Maybe', 'Im', 'in', 'the', 'black', 'maybe', 'Im', 'on', 'my', 'knees', 'Maybe', 'Im', 'in', 'the', 'gap', 'between', 'the', 'two', 'trapezes', 'But', 'my', 'heart', 'i

In [ ]:
# TODO: remove the stop words using NLTK. Then put the result into a variable and print it

# Select the row for the song 'Every Teardrop Is A Waterfall'
song_row = df[df['Song'] == 'Every Teardrop Is A Waterfall']

# Get the lyrics text for the selected song and save it into a variable
lyrics = song_row['Lyrics'].values[0]

# Remove the punctuation from the lyrics
lyrics_no_punct = lyrics.translate(str.maketrans("", "", string.punctuation))

# Tokenize the lyrics without punctuation
tokens = nltk.word_tokenize(lyrics_no_punct)

# Remove the stop words from the tokens
stopwords = nltk.corpus.stopwords.words("english")
tokens_no_stopwords = [token for token in tokens if token.lower() not in stopwords]

# Print the tokens without punctuation and stop words
print(tokens_no_stopwords)

['turn', 'music', 'got', 'records', 'shut', 'world', 'outside', 'lights', 'come', 'Maybe', 'streets', 'alight', 'maybe', 'trees', 'gone', 'feel', 'heart', 'start', 'beating', 'favourite', 'song', 'kids', 'dance', 'kids', 'night', 'Monday', 'morning', 'feels', 'another', 'life', 'turn', 'music', 'Im', 'roll', 'time', 'heaven', 'sight', 'turn', 'music', 'got', 'records', 'underneath', 'rubble', 'sing', 'rebel', 'song', 'Dont', 'want', 'see', 'another', 'generation', 'drop', 'Id', 'rather', 'comma', 'full', 'stop', 'Maybe', 'Im', 'black', 'maybe', 'Im', 'knees', 'Maybe', 'Im', 'gap', 'two', 'trapezes', 'heart', 'beating', 'pulses', 'start', 'Cathedrals', 'heart', 'saw', 'oh', 'light', 'swear', 'emerge', 'blinking', 'tell', 'alright', 'soar', 'walls', 'every', 'siren', 'symphony', 'every', 'tears', 'waterfall', 'waterfall', 'Oh', 'waterfall', 'Oh', 'oh', 'oh', 'waterfall', 'Every', 'tear', 'waterfall', 'Oh', 'oh', 'oh', 'hurt', 'hurt', 'bad', 'still', 'Ill', 'raise', 'flag', 'Oh', 'wa', 'w

In [ ]:
# TODO: Perform lemmatization using WordNetLemmatizer on our tokens

# Select the row for the song 'Every Teardrop Is A Waterfall'
song_row = df[df['Song'] == 'Every Teardrop Is A Waterfall']

# Get the lyrics text for the selected song and save it into a variable
lyrics = song_row['Lyrics'].values[0]

# Remove the punctuation from the lyrics
lyrics_no_punct = lyrics.translate(str.maketrans("", "", string.punctuation))

# Tokenize the lyrics without punctuation and stop words
tokens = nltk.word_tokenize(lyrics_no_punct)
stopwords = nltk.corpus.stopwords.words("english")
tokens_no_stopwords = [token for token in tokens if token.lower() not in stopwords]

# Perform lemmatization on the tokens
lemmatizer = nltk.WordNetLemmatizer()
tokens_lemmatized = [lemmatizer.lemmatize(token) for token in tokens_no_stopwords]

# Print the lemmatized tokens
print(tokens_lemmatized)


['turn', 'music', 'got', 'record', 'shut', 'world', 'outside', 'light', 'come', 'Maybe', 'street', 'alight', 'maybe', 'tree', 'gone', 'feel', 'heart', 'start', 'beating', 'favourite', 'song', 'kid', 'dance', 'kid', 'night', 'Monday', 'morning', 'feel', 'another', 'life', 'turn', 'music', 'Im', 'roll', 'time', 'heaven', 'sight', 'turn', 'music', 'got', 'record', 'underneath', 'rubble', 'sing', 'rebel', 'song', 'Dont', 'want', 'see', 'another', 'generation', 'drop', 'Id', 'rather', 'comma', 'full', 'stop', 'Maybe', 'Im', 'black', 'maybe', 'Im', 'knee', 'Maybe', 'Im', 'gap', 'two', 'trapeze', 'heart', 'beating', 'pulse', 'start', 'Cathedrals', 'heart', 'saw', 'oh', 'light', 'swear', 'emerge', 'blinking', 'tell', 'alright', 'soar', 'wall', 'every', 'siren', 'symphony', 'every', 'tear', 'waterfall', 'waterfall', 'Oh', 'waterfall', 'Oh', 'oh', 'oh', 'waterfall', 'Every', 'tear', 'waterfall', 'Oh', 'oh', 'oh', 'hurt', 'hurt', 'bad', 'still', 'Ill', 'raise', 'flag', 'Oh', 'wa', 'wa', 'wa', 'wa

In [ ]:
# TODO: use the function pos_tag of NLTK to perform POS-tagging and print the result

# Select the row for the song 'Every Teardrop Is A Waterfall'
song_row = df[df['Song'] == 'Every Teardrop Is A Waterfall']

# Get the lyrics text for the selected song and save it into a variable
lyrics = song_row['Lyrics'].values[0]

# Remove the punctuation from the lyrics
lyrics_no_punct = lyrics.translate(str.maketrans("", "", string.punctuation))

# Tokenize the lyrics without punctuation and stop words
tokens = nltk.word_tokenize(lyrics_no_punct)
stopwords = nltk.corpus.stopwords.words("english")
tokens_no_stopwords = [token for token in tokens if token.lower() not in stopwords]

# Perform POS-tagging on the tokens
pos_tags = nltk.pos_tag(tokens_no_stopwords)

# Print the POS-tags
print(pos_tags)

[('turn', 'NN'), ('music', 'NN'), ('got', 'VBD'), ('records', 'NNS'), ('shut', 'VBN'), ('world', 'NN'), ('outside', 'IN'), ('lights', 'NNS'), ('come', 'VBP'), ('Maybe', 'RB'), ('streets', 'NNS'), ('alight', 'VBD'), ('maybe', 'RB'), ('trees', 'NNS'), ('gone', 'VBN'), ('feel', 'JJ'), ('heart', 'NN'), ('start', 'NN'), ('beating', 'VBG'), ('favourite', 'NN'), ('song', 'NN'), ('kids', 'NNS'), ('dance', 'NN'), ('kids', 'NNS'), ('night', 'NN'), ('Monday', 'NNP'), ('morning', 'NN'), ('feels', 'NNS'), ('another', 'DT'), ('life', 'NN'), ('turn', 'NN'), ('music', 'NN'), ('Im', 'NNP'), ('roll', 'NN'), ('time', 'NN'), ('heaven', 'JJ'), ('sight', 'VBD'), ('turn', 'NN'), ('music', 'NN'), ('got', 'VBD'), ('records', 'NNS'), ('underneath', 'IN'), ('rubble', 'JJ'), ('sing', 'VBG'), ('rebel', 'NN'), ('song', 'NN'), ('Dont', 'NNP'), ('want', 'NN'), ('see', 'NN'), ('another', 'DT'), ('generation', 'NN'), ('drop', 'NN'), ('Id', 'NNP'), ('rather', 'RB'), ('comma', 'VBZ'), ('full', 'JJ'), ('stop', 'NN'), ('Ma

In [ ]:
from nltk.corpus import wordnet

def get_wordnet_pos(pos_tag):
    output = np.asarray(pos_tag)
    for i in range(len(pos_tag)):
        if pos_tag[i][1].startswith('J'):
            output[i][1] = wordnet.ADJ
        elif pos_tag[i][1].startswith('V'):
            output[i][1] = wordnet.VERB
        elif pos_tag[i][1].startswith('R'):
            output[i][1] = wordnet.ADV
        else:
            output[i][1] = wordnet.NOUN
    return output

In [ ]:
# TODO: Perform the lemmatization properly


from nltk.stem import WordNetLemmatizer
# Select the row for the song 'Every Teardrop Is A Waterfall'
song_row = df[df['Song'] == 'Every Teardrop Is A Waterfall']

# Get the lyrics text for the selected song and save it into a variable
lyrics = song_row['Lyrics'].values[0]

# Remove the punctuation from the lyrics
lyrics_no_punct = lyrics.translate(str.maketrans("", "", string.punctuation))

# Tokenize the lyrics without punctuation and stop words
tokens = nltk.word_tokenize(lyrics_no_punct)
stopwords = nltk.corpus.stopwords.words("english")
tokens_no_stopwords = [token for token in tokens if token.lower() not in stopwords]

# Perform POS-tagging on the tokens
pos_tags = nltk.pos_tag(tokens_no_stopwords)

# Create a WordNetLemmatizer object
lemmatizer = WordNetLemmatizer()

# Perform lemmatization on the tokens with proper POS tagging
lemmatized_tokens = []
for token, pos in pos_tags:
    if pos.startswith('J'):
        # Adjective
        lemma = lemmatizer.lemmatize(token, pos='a')
    elif pos.startswith('V'):
        # Verb
        lemma = lemmatizer.lemmatize(token, pos='v')
    elif pos.startswith('N'):
        # Noun
        lemma = lemmatizer.lemmatize(token, pos='n')
    elif pos.startswith('R'):
        # Adverb
        lemma = lemmatizer.lemmatize(token, pos='r')
    else:
        # Default to noun
        lemma = lemmatizer.lemmatize(token)
    lemmatized_tokens.append(lemma)

# Print the lemmatized tokens
print(lemmatized_tokens)

['turn', 'music', 'get', 'record', 'shut', 'world', 'outside', 'light', 'come', 'Maybe', 'street', 'alight', 'maybe', 'tree', 'go', 'feel', 'heart', 'start', 'beat', 'favourite', 'song', 'kid', 'dance', 'kid', 'night', 'Monday', 'morning', 'feel', 'another', 'life', 'turn', 'music', 'Im', 'roll', 'time', 'heaven', 'sight', 'turn', 'music', 'get', 'record', 'underneath', 'rubble', 'sing', 'rebel', 'song', 'Dont', 'want', 'see', 'another', 'generation', 'drop', 'Id', 'rather', 'comma', 'full', 'stop', 'Maybe', 'Im', 'black', 'maybe', 'Im', 'knee', 'Maybe', 'Im', 'gap', 'two', 'trapezes', 'heart', 'beating', 'pulse', 'start', 'Cathedrals', 'heart', 'saw', 'oh', 'light', 'swear', 'emerge', 'blink', 'tell', 'alright', 'soar', 'wall', 'every', 'siren', 'symphony', 'every', 'tear', 'waterfall', 'waterfall', 'Oh', 'waterfall', 'Oh', 'oh', 'oh', 'waterfall', 'Every', 'tear', 'waterfall', 'Oh', 'oh', 'oh', 'hurt', 'hurt', 'bad', 'still', 'Ill', 'raise', 'flag', 'Oh', 'wa', 'wa', 'wa', 'wa', 'waa

In [ ]:
# TODO: Perform stemming

from nltk.stem import PorterStemmer

# Select the row for the song 'Every Teardrop Is A Waterfall'
song_row = df[df['Song'] == 'Every Teardrop Is A Waterfall']

# Get the lyrics text for the selected song and save it into a variable
lyrics = song_row['Lyrics'].values[0]

# Remove the punctuation from the lyrics
lyrics_no_punct = lyrics.translate(str.maketrans("", "", string.punctuation))

# Tokenize the lyrics without punctuation and stop words
tokens = nltk.word_tokenize(lyrics_no_punct)
stopwords = nltk.corpus.stopwords.words("english")
tokens_no_stopwords = [token for token in tokens if token.lower() not in stopwords]

# Perform stemming on the tokens
stemmer = PorterStemmer()
stemmed_tokens = [stemmer.stem(token) for token in tokens_no_stopwords]

# Print the stemmed tokens
print(stemmed_tokens)


['turn', 'music', 'got', 'record', 'shut', 'world', 'outsid', 'light', 'come', 'mayb', 'street', 'alight', 'mayb', 'tree', 'gone', 'feel', 'heart', 'start', 'beat', 'favourit', 'song', 'kid', 'danc', 'kid', 'night', 'monday', 'morn', 'feel', 'anoth', 'life', 'turn', 'music', 'im', 'roll', 'time', 'heaven', 'sight', 'turn', 'music', 'got', 'record', 'underneath', 'rubbl', 'sing', 'rebel', 'song', 'dont', 'want', 'see', 'anoth', 'gener', 'drop', 'id', 'rather', 'comma', 'full', 'stop', 'mayb', 'im', 'black', 'mayb', 'im', 'knee', 'mayb', 'im', 'gap', 'two', 'trapez', 'heart', 'beat', 'puls', 'start', 'cathedr', 'heart', 'saw', 'oh', 'light', 'swear', 'emerg', 'blink', 'tell', 'alright', 'soar', 'wall', 'everi', 'siren', 'symphoni', 'everi', 'tear', 'waterfal', 'waterfal', 'oh', 'waterfal', 'oh', 'oh', 'oh', 'waterfal', 'everi', 'tear', 'waterfal', 'oh', 'oh', 'oh', 'hurt', 'hurt', 'bad', 'still', 'ill', 'rais', 'flag', 'oh', 'wa', 'wa', 'wa', 'wa', 'waaterfal', 'wa', 'wa', 'wa', 'wa', 'w

III. Text Similarity

In [ ]:
# import needed libraries
import nltk
import numpy as np
import pandas as pd

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
A = "Outside the classroom, Stallman pursued his studies with even more diligence, rushing off to fulfill his laboratory-assistant duties at Rockefeller University during the week and dodging the Vietnam protesters on his way to Saturday school at Columbia. It was there, while the rest of the Science Honors Program students sat around discussing their college choices, that Stallman finally took a moment to participate in the preclass bull session."
B = "To facilitate the process, AI Lab hackers had built a system that displayed both the source and display modes on a split screen. Despite this innovative hack, switching from mode to mode was still a nuisance."
C = "With no dorm and no dancing, Stallman's social universe imploded. Like an astronaut experiencing the aftereffects of zero-gravity, Stallman found that his ability to interact with nonhackers, especially female nonhackers, had atrophied significantly. After 16 weeks in the AI Lab, the self confidence he'd been quietly accumulating during his 4 years at Harvard was virtually gone."

In [ ]:
# TODO: compute the Jaccard similarities
# Split the sentences

# Compute the intersection and union

# Compute and print the Jaccard Similarity

# Define the three sets of words


# Compute the Jaccard Similarity between sets AB, BC, and AC
jaccard_similarity_AB = len(A.string(B)) / len(A.union(B))
jaccard_similarity_BC = len(B.string(C)) / len(B.union(C))
jaccard_similarity_AC = len(A.string(C)) / len(A.union(C))

# Print the Jaccard Similarity between sets AB, BC, and AC
print("Jaccard Similarity AB: ", jaccard_similarity_AB)
print("Jaccard Similarity BC: ", jaccard_similarity_BC)
print("Jaccard Similarity AC: ", jaccard_similarity_AC)

In [ ]:
# TODO: compute the TF-IDF of A, B and C and the cosine similarities of all possibilities

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity



# Create a TfidfVectorizer object
vectorizer = TfidfVectorizer()

# Compute the TF-IDF matrix
tfidf_matrix = vectorizer.fit_transform([A, B, C])

# Compute the cosine similarities between all pairs
cosine_sim_AB = cosine_similarity(tfidf_matrix[0], tfidf_matrix[1])[0][0]
cosine_sim_BC = cosine_similarity(tfidf_matrix[1], tfidf_matrix[2])[0][0]
cosine_sim_AC = cosine_similarity(tfidf_matrix[0], tfidf_matrix[2])[0][0]

# Print the TF-IDF matrix and cosine similarities
# print("TF-IDF matrix:")
# print(tfidf_matrix.toarray())
print("cos(A, B):", cosine_sim_AB)
print("cos(B, C):", cosine_sim_BC)
print("cos(A, C):", cosine_sim_AC)

cos(A, B): 0.16793269576264072
cos(B, C): 0.13618963113796592
cos(A, C): 0.2850296032333907


III. Text Classification 

In [ ]:
# Import NLTK and all the needed libraries
import nltk
import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
# TODO: Load the dataset in coldplay.csv
data = pd.read_csv('coldplay.csv')

data.head()

,Artist,Song,Link,Lyrics
0,Coldplay,Another's Arms,/c/coldplay/anothers+arms_21079526.html,Late night watching tv \nUsed to be you here ...
1,Coldplay,Bigger Stronger,/c/coldplay/bigger+stronger_20032648.html,I want to be bigger stronger drive a faster ca...
2,Coldplay,Daylight,/c/coldplay/daylight_20032625.html,"To my surprise, and my delight \nI saw sunris..."
3,Coldplay,Everglow,/c/coldplay/everglow_21104546.html,"Oh, they say people come \nThey say people go..."
4,Coldplay,Every Teardrop Is A Waterfall,/c/coldplay/every+teardrop+is+a+waterfall_2091...,"I turn the music up, I got my records on \nI ..."


In [ ]:
# TODO: Explore the data
# Create a DataFrame
data = pd.read_csv('coldplay.csv')

# Print the summary
print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 120 entries, 0 to 119
Data columns (total 4 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   Artist  120 non-null    object
 1   Song    120 non-null    object
 2   Link    120 non-null    object
 3   Lyrics  120 non-null    object
dtypes: object(4)
memory usage: 3.9+ KB
None


In [ ]:
# TODO: Compute a BOW
from sklearn.feature_extraction.text import CountVectorizer

# Select all the rows for Coldplay songs
coldplay_rows = df[df['Artist'] == 'Coldplay']

# Get the lyrics text for all the Coldplay songs and save them into a list
lyrics_list = coldplay_rows['Lyrics'].tolist()

# Create a CountVectorizer object
vectorizer = CountVectorizer()

# Fit and transform the lyrics into a bag-of-words matrix
bow_matrix = vectorizer.fit_transform(lyrics_list)

# Print the shape of the bag-of-words matrix
print(bow_matrix.shape)

(120, 1776)


In [ ]:
# TODO: Create a new dataframe containing the BOW outputs and the corresponding words as columns. And print it

from sklearn.feature_extraction.text import CountVectorizer

# Select all the rows for Coldplay songs
coldplay_rows = df[df['Artist'] == 'Coldplay']

# Get the lyrics text for all the Coldplay songs and save them into a list
lyrics_list = coldplay_rows['Lyrics'].tolist()

# Create a CountVectorizer object
vectorizer = CountVectorizer()

# Fit and transform the lyrics into a bag-of-words matrix
bow_matrix = vectorizer.fit_transform(lyrics_list)

# Get the feature names (i.e., the vocabulary) of the bag-of-words matrix
feature_names = vectorizer.get_feature_names_out()

# Convert the bag-of-words matrix to a dense matrix and create a new dataframe
bow_df = pd.DataFrame(bow_matrix.toarray(), columns=feature_names)

# Print the new dataframe
print(bow_df)

     10  2000  2gether  76543  aaaaaah  aaaaah  aaaah  about  above  achin  \
0     0     0        0      0        0       0      0      0      0      0   
1     0     0        0      0        0       0      0      0      0      0   
2     0     0        0      0        0       0      0      0      0      0   
3     0     0        0      0        0       0      0      0      0      0   
4     0     0        0      0        0       0      0      0      0      0   
..   ..   ...      ...    ...      ...     ...    ...    ...    ...    ...   
115   0     0        0      0        0       0      0      1      2      0   
116   0     0        0      0        0       0      0      0      0      0   
117   0     0        1      0        0       0      0      0      0      0   
118   0     0        0      0        0       0      0      0      0      0   
119   0     0        0      0        0       0      0      0      0      0   

     ...  yellow  yes  yesterday  yet  you  young  your  yours 

In [ ]:
sum_bow = bow_df.sum()
sum_bow.idxmax()

'you'

In [ ]:
# TODO: print the 10 most used word by Coldplay

import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer

# Select all the rows for Coldplay songs
coldplay_rows = df[df['Artist'] == 'Coldplay']

# Get the lyrics text for all the Coldplay songs and save them into a list
lyrics_list = coldplay_rows['Lyrics'].tolist()

# Create a CountVectorizer object
vectorizer = CountVectorizer()

# Fit and transform the lyrics into a bag-of-words matrix
bow_matrix = vectorizer.fit_transform(lyrics_list)

# Get the feature names (i.e., the vocabulary) of the bag-of-words matrix
feature_names = vectorizer.get_feature_names_out()

# Convert the bag-of-words matrix to a dense matrix and create a new dataframe
bow_df = pd.DataFrame(bow_matrix.toarray(), columns=feature_names)

# Print the new dataframe
print(bow_df)

# Create a new dataframe with the word counts and feature names
word_counts_df = pd.DataFrame({'word': feature_names, 'count': bow_matrix.sum(axis=0).tolist()[0]})

# Sort the dataframe by word count in descending order
sorted_word_counts_df = word_counts_df.sort_values('count', ascending=False)

# Print the top 10 most used words by Coldplay
print(sorted_word_counts_df.head(10))

     10  2000  2gether  76543  aaaaaah  aaaaah  aaaah  about  above  achin  \
0     0     0        0      0        0       0      0      0      0      0   
1     0     0        0      0        0       0      0      0      0      0   
2     0     0        0      0        0       0      0      0      0      0   
3     0     0        0      0        0       0      0      0      0      0   
4     0     0        0      0        0       0      0      0      0      0   
..   ..   ...      ...    ...      ...     ...    ...    ...    ...    ...   
115   0     0        0      0        0       0      0      1      2      0   
116   0     0        0      0        0       0      0      0      0      0   
117   0     0        1      0        0       0      0      0      0      0   
118   0     0        0      0        0       0      0      0      0      0   
119   0     0        0      0        0       0      0      0      0      0   

     ...  yellow  yes  yesterday  yet  you  young  your  yours 

IV. Topic Modelling 

In [ ]:
import pandas as pd
df = pd.read_csv('random_headlines.csv')

In [ ]:
df.head(10)

,publish_date,headline_text
0,20120305,ute driver hurt in intersection crash
1,20081128,6yo dies in cycling accident
2,20090325,bumper olive harvest expected
3,20100201,replica replaces northernmost sign
4,20080225,woods targets perfect season
5,20091120,leckie salvages dramatic draw for adelaide
6,20031024,group to gauge rail services future
7,20130304,anti hunting rally still going ahead
8,20081115,dr congo refugees receive first aid
9,20130304,thailand signs agreement with muslim rebels


In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20000 entries, 0 to 19999
Data columns (total 2 columns):
 #   Column         Non-Null Count  Dtype 
---  ------         --------------  ----- 
 0   publish_date   20000 non-null  int64 
 1   headline_text  20000 non-null  object
dtypes: int64(1), object(1)
memory usage: 312.6+ KB


In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
import string

In [ ]:
df['lowercase'] = df['headline_text'].str.lower()
df['tokens'] = df['lowercase'].apply(word_tokenize)
df['no_punctuation'] = df['tokens'].apply(lambda tokens: [token for token in tokens if token not in string.punctuation])
stopwords_set = set(stopwords.words('english'))
df['no_stopwords'] = df['no_punctuation'].apply(lambda tokens: [token for token in tokens if token not in stopwords_set])
stemmer = PorterStemmer()
df['stemmed'] = df['no_stopwords'].apply(lambda tokens: [stemmer.stem(token) for token in tokens])

LookupError: ignored

In [ ]:
df['stemmed']

In [ ]:
!pip install gensim

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer, WordNetLemmatizer
import string
from gensim.corpora import Dictionary
     

In [ ]:
lemmatizer = WordNetLemmatizer()
df['lemmatized'] = df['no_stopwords'].apply(lambda tokens: [lemmatizer.lemmatize(token) for token in tokens])

In [ ]:
# Create a dictionary of the tokens
dictionary = Dictionary(df['stemmed'])

# Filter out rare and common tokens
dictionary.filter_extremes(no_below=5, no_above=0.5)

# Convert each headline to its BOW representation
df['bow'] = df['stemmed'].apply(lambda tokens: dictionary.doc2bow(tokens))

V. Named Entity Recognition

In [ ]:
adafile = "ada_lovelace.txt"

In [ ]:
def clean_file(filename):
    with open(filename, 'r') as file:
        contents = file.read()

    redacted_contents = contents.replace("Ada Lovelace", "[REDACTED]")

    with open(filename, 'w') as file:
        file.write(redacted_contents)

clean_file(adafile)

In [ ]:
import spacy

def identify_entities(filename):
    nlp = spacy.load("en_core_web_sm")

    with open(filename, 'r') as file:
        contents = file.read()

    doc = nlp(contents)

    for entity in doc.ents:
        print(entity.text, entity.label_)

identify_entities(adafile)

Augusta Ada King PERSON
Countess PERSON
Lovelace PERSON
Byron ORG
10 December 1815 DATE
27 November 1852 DATE
English LANGUAGE
Charles Babbage's ORG
the Analytical Engine ORG
first ORDINAL
first ORDINAL
first ORDINAL
one CARDINAL
first ORDINAL
Lovelace PERSON
Mary Somerville PERSON
Charles Babbage PERSON
1833 DATE
Somerville GPE
many years DATE
Andrew Crosse PERSON
David Brewster PERSON
Charles Wheatstone PERSON
Michael Faraday PERSON
Charles Dickens PERSON


In [ ]:
import spacy
from spacy import displacy
from IPython.display import display

def visualize_entities(filename):

    nlp = spacy.load("en_core_web_sm")

    with open(filename, 'r') as file:
        contents = file.read()

    doc = nlp(contents)

    displacy.render(doc, style="ent", jupyter=True)


visualize_entities(adafile)

In [ ]:
import spacy

def replace_name_by_redacted(filename):
    nlp = spacy.load("en_core_web_sm")

    with open(filename, 'r') as file:
        contents = file.read()

    doc = nlp(contents)

    redacted_contents = contents
    for entity in doc.ents:
        if entity.label_ == "PERSON":
            redacted_contents = redacted_contents.replace(entity.text, "[REDACTED]")

    with open(filename, 'w') as file:
        file.write(redacted_contents)

replace_name_by_redacted(adafile)

In [ ]:
import spacy

def make_doc_GDPR_compliant(filename):

    nlp = spacy.load("en_core_web_sm")

    with open(filename, 'r') as file:
        contents = file.read()

    doc = nlp(contents)

    redacted_contents = contents
    for entity in doc.ents:
        if entity.label_ == "PERSON":
            redacted_contents = redacted_contents.replace(entity.text, "[REDACTED]")

    with open(filename, 'w') as file:
        file.write(redacted_contents)

make_doc_GDPR_compliant(adafile)

VI. Exercise

In [ ]:
jobmarket = "job-market.csv"

In [ ]:
jobs_df = pd.read_csv("job-market.csv")
jobs_df.fillna(0)

,Id,Title,Company,Date,Location,Area,Classification,SubClassification,Requirement,FullDescription,LowestSalary,HighestSalary,JobType
0,37404348.0,Casual Stock Replenisher,Aldi Stores,2018-10-07T00:00:00.000Z,Sydney,North West & Hills District,Retail & Consumer Products,Retail Assistants,Our Casual Stock Replenishers pride themselves...,0,0.0,30.0,0
1,37404337.0,Casual Stock Replenisher,Aldi Stores,2018-10-07T00:00:00.000Z,Richmond & Hawkesbury,0,Retail & Consumer Products,Retail Assistants,Our Casual Stock Replenishers pride themselves...,0,0.0,30.0,0
2,37404356.0,RETAIL SALES SUPERSTARS and STYLISTS Wanted - ...,LB Creative Pty Ltd,2018-10-07T00:00:00.000Z,Brisbane,CBD & Inner Suburbs,Retail & Consumer Products,Retail Assistants,BRAND NEW FLAGSHIP STORE OPENING - SUNSHINE PLAZA,0,0.0,30.0,0
3,37404330.0,Team member - Belrose,Anaconda Group Pty Ltd,2018-10-07T00:00:00.000Z,Gosford & Central Coast,0,Retail & Consumer Products,Retail Assistants,Bring it on - do you love the great outdoors a...,0,0.0,30.0,0
4,37404308.0,"Business Banking Contact Centre Specialist, Ni...",Commonwealth Bank - Business & Private Banking,2018-10-07T00:00:00.000Z,Sydney,Ryde & Macquarie Park,Call Centre & Customer Service,Sales - Inbound,"We are seeking highly articulate, enthusiastic...",0,0.0,30.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
40784,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
40785,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
40786,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0
40787,0.0,0,0,0,0,0,0,0,0,0,0.0,0.0,0


In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression

# Step 1: Filter the jobs for the IT sector only
jobs_df['Title'] = jobs_df['Title'].fillna('')
it_jobs_df = jobs_df[jobs_df['Title'].str.contains('IT', case=False)]

In [ ]:
# Step 2: Put the descriptions of all jobs into a list
job_descriptions = it_jobs_df['Classification'].tolist()

In [ ]:
# Step 3: Use scikit-learn to get the top 20 important keywords
vectorizer = TfidfVectorizer(stop_words='english', max_features=1000)
X = vectorizer.fit_transform(job_descriptions)
# Train a logistic regression model
model = LogisticRegression()
model.fit(X, it_jobs_df['Title'])

# Get feature names and their corresponding coefficients
feature_names = vectorizer.get_feature_names_out()
coefficients = model.coef_[0]

# Sort feature names based on coefficients
top_keywords = sorted(zip(coefficients, feature_names), reverse=True)[:20]

# Print the top 20 important keywords
print("Top 20 important keywords:")
for coef, keyword in top_keywords:
    print(keyword)

Top 20 important keywords:
retail
products
consumer
management
general
ceo
strategy
consulting
science
farming
conservation
animals
superannuation
insurance
media
arts
advertising
financial
banking
real


In [ ]:
favorite_keyword = 'python'

# Construct a query using the favorite keyword
query = vectorizer.transform([favorite_keyword])

# Compute similarity scores between the query and job descriptions
similarity_scores = model.predict_proba(query)[0]

# Rank the job descriptions based on similarity scores
ranked_jobs = sorted(zip(similarity_scores, it_jobs_df['Title'], job_descriptions), reverse=True)

# Print the top 5 job descriptions most similar to the query
print("\nTop 5 job descriptions most similar to the query:")
for score, job_title, job_description in ranked_jobs[:5]:
    print("Job Title:", job_title)
    print("Similarity Score:", score)
    print("Job Description:", job_description)
    print()


Top 5 job descriptions most similar to the query:
Job Title: Credit Controller - Temporary Position
Similarity Score: 0.004172072785443803
Job Description: Accounting

Job Title: Wait Staff
Similarity Score: 0.0033997001440222003
Job Description: Hospitality & Tourism

Job Title: Solution Architect (IAM)
Similarity Score: 0.003043823447949755
Job Description: Information & Communication Technology

Job Title: Recruitment Consultant
Similarity Score: 0.0027987719273321284
Job Description: Human Resources & Recruitment

Job Title: IT Security Architect
Similarity Score: 0.002747864116015302
Job Description: Information & Communication Technology



In [ ]:
def extract_ngrams(sequence, n):
    ngrams = []
    sequence_length = len(sequence)
    for i in range(sequence_length - n + 1):
        ngram = sequence[i:i+n]
        ngrams.append(ngram)
    return ngrams

In [ ]:
sentence = "I like deadline and want to immerse myself in deadline."

# Extract word tri-grams
words = sentence.split()
word_trigrams = extract_ngrams(words, 3)

print("Word Tri-grams:")
for trigram in word_trigrams:
    print(trigram)

# Extract letter tri-grams
letters = list(sentence.replace(" ", ""))
letter_trigrams = extract_ngrams(letters, 3)

print("\nLetter Tri-grams:")
for trigram in letter_trigrams:
    print(trigram)

Word Tri-grams:
['I', 'like', 'deadline']
['like', 'deadline', 'and']
['deadline', 'and', 'want']
['and', 'want', 'to']
['want', 'to', 'immerse']
['to', 'immerse', 'myself']
['immerse', 'myself', 'in']
['myself', 'in', 'deadline.']

Letter Tri-grams:
['I', 'l', 'i']
['l', 'i', 'k']
['i', 'k', 'e']
['k', 'e', 'd']
['e', 'd', 'e']
['d', 'e', 'a']
['e', 'a', 'd']
['a', 'd', 'l']
['d', 'l', 'i']
['l', 'i', 'n']
['i', 'n', 'e']
['n', 'e', 'a']
['e', 'a', 'n']
['a', 'n', 'd']
['n', 'd', 'w']
['d', 'w', 'a']
['w', 'a', 'n']
['a', 'n', 't']
['n', 't', 't']
['t', 't', 'o']
['t', 'o', 'i']
['o', 'i', 'm']
['i', 'm', 'm']
['m', 'm', 'e']
['m', 'e', 'r']
['e', 'r', 's']
['r', 's', 'e']
['s', 'e', 'm']
['e', 'm', 'y']
['m', 'y', 's']
['y', 's', 'e']
['s', 'e', 'l']
['e', 'l', 'f']
['l', 'f', 'i']
['f', 'i', 'n']
['i', 'n', 'd']
['n', 'd', 'e']
['d', 'e', 'a']
['e', 'a', 'd']
['a', 'd', 'l']
['d', 'l', 'i']
['l', 'i', 'n']
['i', 'n', 'e']
['n', 'e', '.']


In [ ]:

import random

def modify_phrase(phrase):
    words = phrase.split()
    modified_words = []
    
    for word in words:
        if len(word) <= 4:
            modified_words.append(word)
        else:
            first_letter = word[0]
            last_letter = word[-1]
            middle_letters = list(word[1:-1])
            random.shuffle(middle_letters)
            modified_word = first_letter + ''.join(middle_letters) + last_letter
            modified_words.append(modified_word)
    
    modified_phrase = ' '.join(modified_words)
    return modified_phrase

# Example phrase
phrase = "I couldn't believe that I could completely understand what I was reading: the astounding power of the human mind"

modified_phrase = modify_phrase(phrase)

print("Original phrase:")
print(phrase)
print()
print("Modified phrase:")
print(modified_phrase)

Original phrase:
I couldn't believe that I could completely understand what I was reading: the astounding power of the human mind

Modified phrase:
I clu'ndot blievee that I cloud clpmeotley unasdnterd what I was rniaegd: the aidnosuntg pweor of the haumn mind


In [ ]:
alice = "alice.txt"

In [ ]:
import nltk

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


True

In [ ]:
import nltk

# Read the input file
with open(alice, 'r') as file:
    text = file.read()

# Tokenize the text into sentences
sentences = nltk.sent_tokenize(text)

# Perform POS tagging on each sentence
tagged_sentences = []
for sentence in sentences:
    tagged_sentence = nltk.pos_tag(nltk.word_tokenize(sentence))
    tagged_sentences.append(tagged_sentence)

# Save the POS tagged output to a separate file
output_file = 'alice_pos_tagged.txt'
with open(output_file, 'w') as file:
    for tagged_sentence in tagged_sentences:
        tagged_text = ' '.join([f"{word}/{tag}" for word, tag in tagged_sentence])
        file.write(tagged_text + '\n')

print(f"POS tagged output saved to '{output_file}'.")

POS tagged output saved to 'alice_pos_tagged.txt'.


In [ ]:
# Open the POS tagged file for reading
with open('alice_pos_tagged.txt', 'r') as file:
    pos_tagged_text = file.read()

# Print the contents of the POS tagged file
print("POS tagged text:")
print(pos_tagged_text)


POS tagged text:
ALICE/NNP 'S/POS ADVENTURES/NNP IN/NNP WONDERLAND/NNP Lewis/NNP Carroll/NNP THE/NNP MILLENNIUM/NNP FULCRUM/NNP EDITION/NNP 3.0/CD CHAPTER/NN I/PRP ./.
Down/IN the/DT Rabbit-Hole/JJ Alice/NNP was/VBD beginning/VBG to/TO get/VB very/RB tired/JJ of/IN sitting/VBG by/IN her/PRP$ sister/NN on/IN the/DT bank/NN ,/, and/CC of/IN having/VBG nothing/NN to/TO do/VB :/: once/RB or/CC twice/VB she/PRP had/VBD peeped/VBN into/IN the/DT book/NN her/PRP$ sister/NN was/VBD reading/VBG ,/, but/CC it/PRP had/VBD no/DT pictures/NNS or/CC conversations/NNS in/IN it/PRP ,/, 'and/VB what/WP is/VBZ the/DT use/NN of/IN a/DT book/NN ,/, '/'' thought/JJ Alice/NNP 'without/POS pictures/NNS or/CC conversations/NNS ?/. '/''
So/IN she/PRP was/VBD considering/VBG in/IN her/PRP$ own/JJ mind/NN (/( as/RB well/RB as/IN she/PRP could/MD ,/, for/IN the/DT hot/JJ day/NN made/VBD her/PRP$ feel/JJ very/RB sleepy/JJ and/CC stupid/JJ )/) ,/, whether/IN the/DT pleasure/NN of/IN making/VBG a/DT daisy-chain/NN w